# Exploratory Data Analysis with a small subset of data
- Use a subset of major directors -> around 20
- subset those directors to their films
- subset ratings and etc
- need to explor incorporting OMDB to get the ratings and view various ranking systems on good films


#### Load Packages

In [3]:
import pandas as pd
import polars as pl
import gzip
import pyarrow
import sys
from pathlib import Path
from tqdm import tqdm
import os


import matplotlib.pyplot as plt
import seaborn as sns
import sys
from pathlib import Path

# Add the `src` folder to Python's search path
src_path = "/Users/bradkittrell/Projects/imdb/IMDB_Fun/src"
sys.path.append(src_path)

In [4]:
from data import generate_dataset

## Load Data and merge

In [83]:
# going to use imdbs list of top 25 directors of all time

names = generate_dataset.load_names(None)
film_crew = generate_dataset.load_crew(None)
films = generate_dataset.load_title_basics(None)

INFO:root:No path provided. Using default path: /Users/bradkittrell/Projects/imdb/IMDB_Fun/Data/processed/name.parquet
INFO:root:Successfully loaded data from /Users/bradkittrell/Projects/imdb/IMDB_Fun/Data/processed/name.parquet
INFO:root:Filtered names data to include only rows where birthYear > 1899
INFO:root:Dropped __index_level_0__ column
INFO:root:No path provided. Using default path: /Users/bradkittrell/Projects/imdb/IMDB_Fun/Data/processed/title_crew.parquet
INFO:root:Successfully loaded data from /Users/bradkittrell/Projects/imdb/IMDB_Fun/Data/processed/title_crew.parquet
INFO:root:No path provided. Using default path: /Users/bradkittrell/Projects/imdb/IMDB_Fun/Data/processed/title_basics_movies.parquet
INFO:root:Successfully loaded data from /Users/bradkittrell/Projects/imdb/IMDB_Fun/Data/processed/title_basics_movies.parquet
INFO:root:Dropped __index_level_0__ column


In [84]:

top_n_directors = ["Steven Spielberg", "Martin Scorsese", "Alfred Hitchcock", "Roman Polanksi",
                   "John Ford", "Billy Wilder", "James Cameron", "Tim Burton", "Ridley Scott", "Joel Coen", "David Fincher",
                   "Christopher Nolan", "Clint Eastwood", "David Lean", "Woody Allen", "Stanley Kubrick", "Danny Boyle", "George Lucas", "Peter Jackson", "Quentin Tarantino",
                   "Fancis Ford Coppola"]
# lots of duplicate names or same names rather
top_n_directors_names = names.filter((pl.col("primaryName").is_in(
    top_n_directors)) & (pl.col("nconst") != "nm13407303") & (pl.col("nconst") != "nm0131607") & (pl.col("nconst") != "nm0131609") & (pl.col("nconst") != "nm7812401"))[["nconst", "primaryName"]]

top_n_directors_names


film_crew = film_crew.filter(
    pl.col("directors").is_in(top_n_directors_names["nconst"]))

top_n_directors_names_films = film_crew.join(top_n_directors_names, left_on="directors",
                                             right_on="nconst", how="inner")[["tconst", "primaryName"]]

top_n_directors_names_films = top_n_directors_names_films.join(
    films, on="tconst")

In [ ]:
import requests

#  http://www.omdbapi.com/?i=tt3896198&apikey=95497563


def fetch_movie_data(title, api_key):

    url = f"http://www.omdbapi.com/?i={title}&plot=full&apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None


movie_data = fetch_movie_data('tt7131622', '95497563')
# print(movie_data)

http://www.omdbapi.com/?i=tt7131622&plot=full&apikey=95497563


In [114]:
from urllib import response


test_df = pd.DataFrame(movie_data)


def response_to_df(response):

    _ = pd.DataFrame(response)
    _['RatingSource'] = _["Ratings"].apply(lambda x: x.get('Source'))
    _['RatingValue'] = _["Ratings"].apply(lambda x: x.get('Value'))

    return _


response_to_df(movie_data).to_csv(
    "/Users/bradkittrell/Projects/imdb/IMDB_Fun/Data/external/omdb.csv", index=False)

In [90]:
top_n_directors_names_films.tail(10)

tconst,primaryName,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
str,str,str,str,str,i64,i64,i64,i64,str
"""tt6019206""","""Quentin Tarantino""","""movie""","""Kill Bill: The Whole Bloody Af…","""Kill Bill: The Whole Bloody Af…",0,2006,0,247,"""Action,Crime,Thriller"""
"""tt6294822""","""Steven Spielberg""","""movie""","""The Post""","""The Post""",0,2017,0,116,"""Biography,Drama,History"""
"""tt6723592""","""Christopher Nolan""","""movie""","""Tenet""","""Tenet""",0,2020,0,150,"""Action,Sci-Fi,Thriller"""
"""tt6802308""","""Clint Eastwood""","""movie""","""The 15:17 to Paris""","""The 15:17 to Paris""",0,2018,0,94,"""Biography,Drama,Thriller"""
"""tt7131622""","""Quentin Tarantino""","""movie""","""Once Upon a Time... in Hollywo…","""Once Upon a Time in... Hollywo…",0,2019,0,161,"""Comedy,Drama"""
"""tt7139936""","""Woody Allen""","""movie""","""A Rainy Day in New York""","""A Rainy Day in New York""",0,2019,0,92,"""Comedy,Romance"""
"""tt7959026""","""Clint Eastwood""","""movie""","""The Mule""","""The Mule""",0,2018,0,116,"""Crime,Drama,Thriller"""
"""tt8079248""","""Danny Boyle""","""movie""","""Yesterday""","""Yesterday""",0,2019,0,116,"""Comedy,Fantasy,Music"""
"""tt8593904""","""Woody Allen""","""movie""","""Rifkin's Festival""","""Rifkin's Festival""",0,2020,0,88,"""Comedy,Romance"""
